In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from ema_workbench import load_results
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, 
                           ema_logging, Constant, Scenario, Policy, CategoricalParameter, IntegerParameter,
                           perform_experiments, SequentialEvaluator, Constraint)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from ema_workbench.util import ema_logging, utilities
import time
from problem_formulation import get_model_for_problem_formulation
#from signal import signal, SIGPIPE, SIG_DFL
#signal(SIGPIPE, SIG_DFL)
import pickle
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [3]:
# The scenarios are saved in a pickle
with open("results/results_deep_pickle_1000nfe_D", "rb") as fp:   # Unpickling
    results = pickle.load(fp)

In [4]:
# concatenate all scenario dataframes into one and add a column that shows from which scenario it was
frames = []
for i in range(len(results)):
    results[i]["scenario"] = i
    frames.append(results[i])
results_fin = pd.concat(frames)
results_fin

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,All Costs,Expected Number of Deaths,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,scenario
0,0,1,0,0,0,0,0,0,0,0,...,3,8,3,1,5.956749e+08,0.008392,0.000000e+00,1.190769e+08,0.000000,0
1,1,1,0,0,0,0,1,1,0,1,...,5,10,1,0,8.760807e+08,0.000089,0.000000e+00,2.495376e+08,0.000000,0
2,1,1,0,0,0,0,0,1,1,0,...,7,9,3,5,7.350769e+08,0.001011,0.000000e+00,6.813900e+07,0.000000,0
3,0,1,0,0,1,0,0,1,1,0,...,3,10,7,0,7.837718e+08,0.000092,0.000000e+00,6.813900e+07,0.000000,0
4,1,1,1,0,0,0,0,1,0,0,...,4,10,1,2,9.537893e+08,0.000000,0.000000e+00,5.397251e+07,0.000000,0
5,1,1,0,0,0,0,0,1,0,0,...,2,10,3,1,6.020952e+08,0.001076,0.000000e+00,6.813900e+07,0.000000,0
6,1,1,0,0,0,0,0,1,1,0,...,3,10,4,5,7.564883e+08,0.000127,0.000000e+00,6.071731e+07,0.000000,0
0,0,1,0,0,0,0,1,1,1,1,...,1,6,4,3,8.790271e+08,0.000204,0.000000e+00,6.813900e+07,0.000000,1
1,0,0,0,0,0,0,1,1,1,0,...,5,10,6,3,6.662139e+08,0.000000,0.000000e+00,1.152729e+08,0.000000,1
2,0,0,1,0,0,0,1,1,1,1,...,5,9,6,3,8.992096e+08,0.000000,0.000000e+00,5.397251e+07,0.000000,1


In [5]:
results_fin.reset_index(inplace=True)
results_fin.drop(["index"],axis=1,inplace=True)
results_fin

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,All Costs,Expected Number of Deaths,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,scenario
0,0,1,0,0,0,0,0,0,0,0,...,3,8,3,1,5.956749e+08,0.008392,0.000000e+00,1.190769e+08,0.000000,0
1,1,1,0,0,0,0,1,1,0,1,...,5,10,1,0,8.760807e+08,0.000089,0.000000e+00,2.495376e+08,0.000000,0
2,1,1,0,0,0,0,0,1,1,0,...,7,9,3,5,7.350769e+08,0.001011,0.000000e+00,6.813900e+07,0.000000,0
3,0,1,0,0,1,0,0,1,1,0,...,3,10,7,0,7.837718e+08,0.000092,0.000000e+00,6.813900e+07,0.000000,0
4,1,1,1,0,0,0,0,1,0,0,...,4,10,1,2,9.537893e+08,0.000000,0.000000e+00,5.397251e+07,0.000000,0
5,1,1,0,0,0,0,0,1,0,0,...,2,10,3,1,6.020952e+08,0.001076,0.000000e+00,6.813900e+07,0.000000,0
6,1,1,0,0,0,0,0,1,1,0,...,3,10,4,5,7.564883e+08,0.000127,0.000000e+00,6.071731e+07,0.000000,0
7,0,1,0,0,0,0,1,1,1,1,...,1,6,4,3,8.790271e+08,0.000204,0.000000e+00,6.813900e+07,0.000000,1
8,0,0,0,0,0,0,1,1,1,0,...,5,10,6,3,6.662139e+08,0.000000,0.000000e+00,1.152729e+08,0.000000,1
9,0,0,1,0,0,0,1,1,1,1,...,5,9,6,3,8.992096e+08,0.000000,0.000000e+00,5.397251e+07,0.000000,1


### Reduce size of set candidate solutions

The set currently contains 35 candidate solutions. We want to reduce this to +- 10 candidate policies. This is more interpretable and less computationally expensive. We can include the policies that score better on the outcomes of interest. We want to maintain diversity, so we want to include at least one policy from each scenario in the ultimate set of policies. 

In [6]:
#We see that there are two candidate policies where people in dikering 1 die
policies = results_fin
policies = policies[policies["A.1_Expected Number of Deaths"] == 0]
policies = policies[policies["A.1_Expected Annual Damage"] == 0]
policies

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,All Costs,Expected Number of Deaths,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,scenario
0,0,1,0,0,0,0,0,0,0,0,...,3,8,3,1,5.956749e+08,0.008392,0.0,1.190769e+08,0.0,0
1,1,1,0,0,0,0,1,1,0,1,...,5,10,1,0,8.760807e+08,0.000089,0.0,2.495376e+08,0.0,0
2,1,1,0,0,0,0,0,1,1,0,...,7,9,3,5,7.350769e+08,0.001011,0.0,6.813900e+07,0.0,0
3,0,1,0,0,1,0,0,1,1,0,...,3,10,7,0,7.837718e+08,0.000092,0.0,6.813900e+07,0.0,0
4,1,1,1,0,0,0,0,1,0,0,...,4,10,1,2,9.537893e+08,0.000000,0.0,5.397251e+07,0.0,0
5,1,1,0,0,0,0,0,1,0,0,...,2,10,3,1,6.020952e+08,0.001076,0.0,6.813900e+07,0.0,0
6,1,1,0,0,0,0,0,1,1,0,...,3,10,4,5,7.564883e+08,0.000127,0.0,6.071731e+07,0.0,0
7,0,1,0,0,0,0,1,1,1,1,...,1,6,4,3,8.790271e+08,0.000204,0.0,6.813900e+07,0.0,1
8,0,0,0,0,0,0,1,1,1,0,...,5,10,6,3,6.662139e+08,0.000000,0.0,1.152729e+08,0.0,1
9,0,0,1,0,0,0,1,1,1,1,...,5,9,6,3,8.992096e+08,0.000000,0.0,5.397251e+07,0.0,1


In [7]:
policies = policies[policies["A.1_Dike Investment Costs"] < results_fin["A.1_Dike Investment Costs"].quantile(0.4)]
policies = policies[policies["Expected Number of Deaths"] < results_fin["Expected Number of Deaths"].quantile(0.5)]

In [8]:
policies

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,All Costs,Expected Number of Deaths,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,scenario
4,1,1,1,0,0,0,0,1,0,0,...,4,10,1,2,9.537893e+08,0.000000,0.0,5.397251e+07,0.0,0
9,0,0,1,0,0,0,1,1,1,1,...,5,9,6,3,8.992096e+08,0.000000,0.0,5.397251e+07,0.0,1
11,0,1,1,0,0,0,1,1,0,1,...,4,9,6,6,8.663231e+08,0.000227,0.0,4.229151e+07,0.0,1
15,0,0,0,0,0,1,1,1,1,1,...,5,10,6,0,8.134500e+08,0.000309,0.0,5.397251e+07,0.0,1
21,0,0,1,0,0,0,1,1,0,0,...,7,6,5,7,5.828116e+08,0.000000,0.0,4.784795e+07,0.0,2
23,0,1,0,0,0,0,1,0,0,0,...,1,6,2,3,5.339177e+08,0.000113,0.0,5.397251e+07,0.0,2
27,0,0,0,0,0,0,0,0,1,0,...,5,6,3,1,3.699433e+08,0.000000,0.0,4.229151e+07,0.0,3
33,0,1,0,0,0,0,0,1,1,0,...,5,9,2,3,4.919386e+08,0.000072,0.0,3.725520e+07,0.0,4


In [9]:
# Check for duplicate policies
policies.iloc[:,:-5].duplicated().value_counts()

False    8
dtype: int64

In [10]:
#Only include the policy options
policies = policies.iloc[:,:-6]
policies

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2
4,1,1,1,0,0,0,0,1,0,0,...,1,9,1,2,6,3,4,10,1,2
9,0,0,1,0,0,0,1,1,1,1,...,5,10,4,9,6,7,5,9,6,3
11,0,1,1,0,0,0,1,1,0,1,...,4,10,2,7,7,1,4,9,6,6
15,0,0,0,0,0,1,1,1,1,1,...,6,9,2,3,6,4,5,10,6,0
21,0,0,1,0,0,0,1,1,0,0,...,2,8,1,3,1,4,7,6,5,7
23,0,1,0,0,0,0,1,0,0,0,...,2,9,3,5,1,4,1,6,2,3
27,0,0,0,0,0,0,0,0,1,0,...,0,7,2,4,7,0,5,6,3,1
33,0,1,0,0,0,0,0,1,1,0,...,0,9,0,4,6,3,5,9,2,3


In [11]:
from ema_workbench import Policy

policies_to_evaluate = []

for i, policy in policies.iterrows():
    policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [13]:
outcome_ranges = pd.read_csv("results/outcome_ranges.csv")
outcome_ranges.set_index("Unnamed: 0",inplace=True)


from problem_formulation_modified import get_model_for_problem_formulation_modified
dike_model,planning_steps = get_model_for_problem_formulation_modified(outcome_ranges)

n_scenarios = 1000
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies_to_evaluate)

[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 1000 scenarios * 8 policies * 1 model(s) = 8000 experiments
100%|████████████████████████████████████| 8000/8000 [2:17:32<00:00,  1.03s/it]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [28]:
experiments, outcomes = results

In [34]:
type(outcomes)

dict

In [37]:
outcomes_df = pd.DataFrame.from_dict(outcomes)

In [39]:
results_ = experiments.join(outcomes_df)

In [41]:
results_.to_csv("results/results_reevaluation.csv")